In [ ]:
!pip -q install langchain-community langchain_openai langchain_chroma langchain pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader= PyPDFLoader('/content/Cloudflare_DPA.pdf')
pdf_data= loader.load()

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter= CharacterTextSplitter(
    separator= '\n',
    chunk_size= 1000,
    chunk_overlap= 200,
    length_function= len,
)
docs= text_splitter.split_documents(pdf_data)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


open_ai_key = "openAI key"
embeddings= OpenAIEmbeddings(api_key= open_ai_key)
db= Chroma.from_documents(docs, embeddings)

In [ ]:
retriver = db.as_retriever(search_kwargs={'k': 3})

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ChatMessageHistory

message_history = ChatMessageHistory()
memory= ConversationBufferMemory(
    memory_key= "chat_history",
    output_key= 'answer',
    chat_memory= message_history,
    return_messages= True,
)



In [ ]:
model= ChatOpenAI(api_key= open_ai_key)

Custom_prompt_template =  """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer
Context : {context}
Question : {question}
only return the helpful answer below and nothing else.
Helpful answer :
"""
PROMPT = PromptTemplate(
    template=Custom_prompt_template, input_variables=["context", "question"]
)

qa =ConversationalRetrievalChain.from_llm(
    llm= model,
    chain_type= 'stuff',
    retriever= retriver,
    memory= memory,
    return_source_documents= True,
    combine_docs_chain_kwargs={'prompt': PROMPT},
)

In [ ]:
question = " is Claudflaire GDRP complaint"
result = qa(question)
result["answer"]